## **Constraints**

A Jupyter notebook talking about parameter constraints in PHOEBE. This roughly follows the given tutorial provided at https://phoebe-project.org/docs/2.4/.

### Setup

Let's quickly install PHOEBE (if needed), load it and other librarys, set up the logger and load the default binary Bundle.

In [ ]:
# !pip install phoebe

In [ ]:
import phoebe as phb
import numpy as np
import matplotlib.pyplot as plt

In [2]:
logger = phb.logger()
bSystem = phb.default_binary()

### What are Constraints?

Constraints pair up with parameters and are typically created by default. They can be listed by using `bSystem.filter(context='constraint')` or `bSystem['constraint']`.

In [3]:
bSystem.filter(context='constraint')

<ParameterSet: 31 parameters | kinds: extinction, star, orbit>

In [4]:
bSystem['constraint']

<ParameterSet: 31 parameters | kinds: extinction, star, orbit>

You can access a constraint like accessing any parameter. Let us get the mass of the primary star.

In [5]:
print(bSystem['mass@primary@constraint'])

Constrains (qualifier): mass
Expression in solar units (value): (3.94784176043574320e+01 * ({sma@binary@component} ** 3.000000)) / ((({period@binary@component} ** 2.000000) * ({q@binary@component} + 1.000000)) * 2.94220621750441933e+03)
Current Result (result): 0.9988131257959815 solMass


The parameter is aware of its constraint, and will reference all the relavent linking parameters when you call `bSystem.get_parameter()` or use a twig.

In [6]:
primaryMass = bSystem['mass@primary@component']
print(primaryMass)

Parameter: mass@primary@component
                       Qualifier: mass
                     Description: Mass
                           Value: 0.9988131257959815 solMass
                  Constrained by: sma@binary@component, period@binary@component, q@binary@component
                      Constrains: logg@primary@component, mass@secondary@component
                      Related to: requiv@primary@component, logg@primary@component, sma@binary@component, period@binary@component, q@binary@component, mass@secondary@component



You can also use `bSystem.constrained_by` to see the relavent parameters that constrain it.

In [7]:
print(primaryMass.constrained_by)

[<Parameter: sma=5.3 solRad | keys: description, value, quantity, default_unit, limits, visible_if, copy_for, readonly, advanced, latexfmt>, <Parameter: period=1.0 d | keys: description, value, quantity, default_unit, limits, visible_if, copy_for, readonly, advanced, latexfmt>, <Parameter: q=1.0 | keys: description, value, quantity, default_unit, limits, visible_if, copy_for, readonly, advanced, latexfmt>]


### Re-Parameterizing / Flipping Constraints

**Note:** When re-parameterizing, be careful to make sure results and parameters make sense.

Constrained parameters are derived from other parameters. However, this can be reversed, setting the constrained parameter in return for constraining one of its related parameters. 

From above, the mass is constrained by the semi-major axis, the period and the mass ratio. Let's instead set the mass by using `bSystem.flip_constraint`, and have the period be constrained.

In [8]:
bSystem.flip_constraint(
    'mass@primary', # Target parameter
    solve_for='period' # New parameter to be constrained - MUST be originally constraining the target parameter
)

<ConstraintParameter: {period@binary@component} = ((3.94784176043574320e+01 * ({sma@binary@component} ** 3.000000)) / (({mass@primary@component} * ({q@binary@component} + 1.000000)) * 2.94220621750441933e+03)) ** (1./2) (solar units) => 1.0 d>

In [ ]:
print('Original Period: {}'.format(bSystem['period@binary@component'].get_value()))

bSystem['mass@primary@component'] = 1 # Set mass of primary to 1 solar mass, from 0.9988...
print('Changed Period: {}'.format(bSystem['period@binary@component'].get_value()))

Original Period: 1.0
Changed Period: 0.9994063867096215


In this example, the secondary mass has also changed due to its relation with the mass ratio `q`. As `q` remains constant but the primary mass has changed, the secondary mass changes to reflect this. 

Keep this in mind that when flipping constraints, changing the parameter might also change other parameters that you didn't explicity say to constrain with `bSystem.flip_constraint`.

## Built-In Constraints

There are different constraints that are default. The full list is below.

### asini
Handles projected semi-major axis along the line of sight, which can be inverted to solve for either 'asini', semi-major axis or inclination.

### esinw, ecosw
Handles projected eccentricity, good representation of the geometry of a lightcurve. Can be inverted for eccentricity or 'per0'.

### t0
Handles converting between t0 conversions - namely providing a reference time at periastron passage (t0_perpass) and at superior conjuction (t0_supconj).

### freq
Handles converting from frequency to period, which can be inverted to solve for period.

### mass
Handles solving for the mass of a component by obeying Kepler 3. Can be inverted to solve for the semi-major axis, mass ratio or period (in addition to mass).

### component sma

Handles computing the semi-major axis of a component about the centre of mass of the parent orbit. This is not the same as the semi-major axis of the parent orbit. Can be inverted to solve for the semi-major axis of the parent body, but not the mass ratio.

### component asini
Handles computing the projected semi-major axis of a component about the centre of mass of the parent orbit. This is not the same as the asini of the parent orbit. Can be inverted to solve for the semi-major axis of the parent body.

### requiv_max
Solves the maxium equivalent radius (for a detached system). A semi-detached system has the radius constrained to this value.

### rotation period
Handles computing the rotation period of a star givien its synchronicty parameter (syncpar). Can be inverted to solve the period (rotational + orbital) and syncpar.

### pitch/yaw (incl/long_an)
Pitch contrains the relation between the orbital and rotational inclination.
Yaw constrains the relation between the orbital and rotational long_an.
If both are 0, the system is aligned.